<a href="https://colab.research.google.com/github/Ana-Januario/Ana-Januario/blob/main/Data_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instead of using fine-tuning, we'll use RAG to build our own "Commander Data" based on everything he ever said in the scripts.

To summarize the high level approach:

- We'll first parse all of the scripts to extract every line from Data, much as we did in the fine-tuning example.
- Then we'll use the OpenAI embeddings API to compute embedding vectors for every one of his lines. This basically gives us similarity measures between every line.
- RAG calls for use of a vector database to store these lines with the associated embedding vectors. To keep things simple, we'll use a local database called vectordb. There are plenty of cloud-based vector database services out there as well.
- Then we'll make a little retrieval function that retrieves the N most-similar lines from the vector database for a given query
- Those similar lines are then added as context to the prompt before it is handed off the the chat API.

I'm intentionally not using langchain or some other higher-level framework, because this is actually pretty simple without it.

First, let's install vectordb:

In [8]:
!pip install vectordb

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of types-requests to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort thi

Now we'll parse out all of the scripts and extract every line of dialog from "DATA". This is almost exactly the same code as from our fine tuning example's preprocessing script. Note you will need to upload all of the script files into a tng folder within your sample_data folder in your CoLab workspace first.

An archive can be found at https://www.st-minutiae.com/resources/scripts/ (look for "All TNG Epsiodes"), but you could easily adapt this to read scripts from your favorite character from your favorite TV show or movie instead.

In [2]:
import os
import re
import random

dialogues = []

def strip_parentheses(s):
    return re.sub(r'\(.*?\)', '', s)

def is_single_word_all_caps(s):
    # First, we split the string into words
    words = s.split()

    # Check if the string contains only a single word
    if len(words) != 1:
        return False

    # Make sure it isn't a line number
    if bool(re.search(r'\d', words[0])):
        return False

    # Check if the single word is in all caps
    return words[0].isupper()

def extract_character_lines(file_path, character_name):
    lines = []
    with open(file_path, 'r') as script_file:
        try:
          lines = script_file.readlines()
        except UnicodeDecodeError:
          pass

    is_character_line = False
    current_line = ''
    current_character = ''
    for line in lines:
        strippedLine = line.strip()
        if (is_single_word_all_caps(strippedLine)):
            is_character_line = True
            current_character = strippedLine
        elif (line.strip() == '') and is_character_line:
            is_character_line = False
            dialog_line = strip_parentheses(current_line).strip()
            dialog_line = dialog_line.replace('"', "'")
            if (current_character == 'DATA' and len(dialog_line)>0):
                dialogues.append(dialog_line)
            current_line = ''
        elif is_character_line:
            current_line += line.strip() + ' '

def process_directory(directory_path, character_name):
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if os.path.isfile(file_path):  # Ignore directories
            extract_character_lines(file_path, character_name)



In [5]:
process_directory("./tng", 'DATA')

Let's do a little sanity check to make sure the lines imported correctly, and print out the first one.

In [6]:
print (dialogues[0])

There is nothing wrong with the Transporter. I have run a complete diagnostic and checked all the targeting components.


Now we'll set up what a document in our vector database looks like; it's just a string and its embedding vector. We're going with 128 dimensions in our embeddings to keep it simple, but you could go larger. The OpenAI model we're using has up to 1536 I believe.

In [9]:
from docarray import BaseDoc
from docarray.typing import NdArray

embedding_dimensions = 128

class DialogLine(BaseDoc):
  text: str = ''
  embedding: NdArray[embedding_dimensions]

It's time to start computing embeddings for each line in OpenAI, so let's make sure OpenAI is installed:

In [10]:
!pip install openai --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.9/389.9 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


Let's initialize the OpenAI client, and test creating an embedding for a single line of dialog just to make sure it works.

You will need to provide your own OpenAI secret key here. To use this code as-is, click on the little key icon in CoLab and add a "secret" for OPENAI_API_KEY that points to your secret key.

In [11]:
from google.colab import userdata

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

embedding_model = "text-embedding-3-small"

response = client.embeddings.create(
    input=dialogues[1],
    dimensions=embedding_dimensions,
    model= embedding_model
)

print(response.data[0].embedding)

[-0.03758013993501663, 0.08628898113965988, -0.10480914264917374, 0.07370690256357193, -0.12482419610023499, -0.017160212621092796, 0.01953751966357231, 0.026264572516083717, 0.012488648295402527, -0.05294439569115639, -0.13171735405921936, 0.15854251384735107, 0.06440529972314835, -0.008756587281823158, -0.00834133755415678, -0.01131556648761034, 0.08267630636692047, -0.14351046085357666, -0.09924478828907013, 0.08587373048067093, 0.10107188671827316, 0.10082273930311203, 0.06369937211275101, 0.01558226253837347, -0.07416367530822754, -0.1201733946800232, 0.09284993261098862, -0.026368385180830956, 0.04044536501169205, 0.04385041818022728, 0.17955416440963745, -0.06781034916639328, 0.09010928124189377, -0.16095095872879028, -0.12075474858283997, 0.011564716696739197, -0.11228363960981369, 0.09550753980875015, 0.0169006809592247, 0.006291039753705263, 0.10754979401826859, -0.022651895880699158, -0.10306508839130402, 0.07096625119447708, 0.03432042524218559, -0.12822924554347992, -0.101

Let's double check that we do in fact have embeddings of 128 dimensions as we specified.

In [12]:
print(len(response.data[0].embedding))

128


OK, now let's compute embeddings for every line Data ever said. The OpenAI API currently can't handle computing them all at once, so we're breaking it up into 128 lines at a time here.

In [13]:
#Generate embeddings for everything Data ever said, 128 lines at a time.
embeddings = []

for i in range(0, len(dialogues), 128):
  dialog_slice = dialogues[i:i+128]
  slice_embeddings = client.embeddings.create(
    input=dialog_slice,
    dimensions=embedding_dimensions,
    model=embedding_model
  )

  embeddings.extend(slice_embeddings.data)

Let's check how many embeddings we actually got back in total.

In [14]:
print (len(embeddings))

6502


Now let's insert every line and its embedding vector into our vector database. The syntax here will vary depending on which vector database you are using, but it's pretty simple.

With vectordb, we need to set up a workspace folder for it to use, so be sure to create that within your CoLab drive first.

In [15]:
from docarray import DocList
import numpy as np
from vectordb import InMemoryExactNNVectorDB

# Specify your workspace path
db = InMemoryExactNNVectorDB[DialogLine](workspace='./sample_data/workspace')

# Index our list of documents
doc_list = [DialogLine(text=dialogues[i], embedding=embeddings[i].embedding) for i in range(len(embeddings))]
db.index(inputs=DocList[DialogLine](doc_list))


WARNING - docarray - Index file does not exist: /content/sample_data/workspace/InMemoryExactNNIndexer[DialogLine][DialogLineWithMatchesAndScores]/index.bin. Initializing empty InMemoryExactNNIndex.


<DocList[DialogLine] (length=6502)>

Let's try querying our vector database for lines similar to a query string.

First we need to compute the embedding vector for our query string, then we'll query the vector database for the top 10 matches based on the similarities encoded by their embedding vectors.

In [16]:
# Perform a search query
queryText = 'Lal, my daughter'
response = client.embeddings.create(
    input=queryText,
    dimensions=embedding_dimensions,
    model=embedding_model
)
query = DialogLine(text=queryText, embedding=response.data[0].embedding)
results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

# Print out the matches
for m in results[0].matches:
  print(m)

DialogLine(
    id='1b3de93db6a705e0b20ef69217ebac85',
    text='That is Lal, my daughter.',
    embedding=NdArray([ 1.27265751e-01, -2.11700285e-03, -1.36252213e-02,
          1.58263575e-02,  5.45372665e-02, -2.06834134e-02,
          1.56717319e-02,  8.31702426e-02, -1.62265643e-01,
          1.61083210e-02,  3.72374207e-02, -1.14313602e-01,
         -4.84250188e-02, -2.47546062e-01,  3.90747339e-02,
          4.99894619e-02,  2.99245510e-02, -1.34687766e-01,
         -1.76527560e-01,  1.35560945e-01,  1.45020381e-01,
          3.50180902e-02,  6.63979352e-02, -7.41837770e-02,
          6.95995912e-02, -2.22160225e-03,  1.92681357e-01,
         -7.92409405e-02, -3.14344205e-02, -6.27596974e-02,
          7.94592351e-02, -1.12858303e-01, -3.74011397e-02,
         -3.84198502e-02, -8.36795941e-02,  9.28479657e-02,
         -7.23100826e-03, -2.10108552e-02, -4.71152514e-02,
         -8.76816586e-02, -1.85986996e-01, -1.15696132e-01,
          5.83210401e-02,  6.38875514e-02,  6.9745123

DialogLine(
    id='c5fccb716877bf71e32c025d2631a1ba',
    text='Lal...',
    embedding=NdArray([ 0.12624386, -0.00786039,  0.0723554 , -0.01190999,  0.08779759,
         -0.08612601,  0.07199721,  0.05364966, -0.12719904,  0.05285367,
          0.01209903, -0.15458107, -0.00163675, -0.08238486,  0.09360831,
          0.01698442,  0.02431746, -0.08011629, -0.08644441,  0.17511758,
          0.09002636,  0.09177753, -0.05396806, -0.01933258,  0.12536827,
          0.01859629,  0.09520029, -0.01391986,  0.04029695,  0.01715356,
         -0.00565152, -0.10984649, -0.0086912 , -0.10674213, -0.08413604,
          0.17177443,  0.02738202, -0.11191606, -0.06001757, -0.08222567,
         -0.19979325, -0.12759703,  0.0654303 ,  0.07279319,  0.1015682 ,
          0.05635602, -0.23131441,  0.03973975, -0.01740231,  0.0578684 ,
         -0.09615547,  0.07339019,  0.09249392,  0.2098227 , -0.07645474,
         -0.0181187 , -0.01944203,  0.10745852,  0.04740115, -0.01587003,
         -0.11621439, -0

DialogLine(
    id='3981c3142c4072e703d3914395bce401',
    text='What do you feel, Lal?',
    embedding=NdArray([ 0.12465294, -0.05501634, -0.01713312,  0.04995255,  0.13775025,
         -0.00806208,  0.01483918, -0.00800973, -0.04306123,  0.10569229,
          0.00161813, -0.10675836, -0.10995653, -0.13280068,  0.00802401,
          0.00701982,  0.06765677, -0.1011996 , -0.11102259,  0.0690655 ,
          0.11292627,  0.01652394, -0.07744168, -0.02025515,  0.11033727,
         -0.15671091,  0.09960052,  0.03504674, -0.04945759,  0.03036369,
         -0.02910726, -0.10500697, -0.02423384, -0.14848702, -0.09815372,
          0.06209803, -0.0683421 , -0.0249382 ,  0.00796214, -0.11543913,
         -0.21153688, -0.08063987,  0.05193238,  0.09145277,  0.08863533,
         -0.0198173 , -0.17072199,  0.05139935, -0.05947095,  0.0353894 ,
         -0.03557977,  0.01050831,  0.10531156,  0.2812877 , -0.12061714,
         -0.0105559 , -0.07401507,  0.08482797,  0.06586731,  0.02851712,
        

DialogLine(
    id='db76d73cd8e72e946ad345fdf8b7ca1f',
    text='Yes, Lal. I am here.',
    embedding=NdArray([ 0.02406189,  0.0076022 , -0.02174187, -0.08027258,  0.14105701,
          0.01713498, -0.00146865,  0.07795256, -0.12296089,  0.03393853,
         -0.04454432, -0.07311367, -0.0004552 , -0.09644641, -0.0334911 ,
          0.08869093, -0.03317624,  0.06751249, -0.10652191,  0.17884015,
          0.05631012, -0.06787706, -0.03364024,  0.0674462 ,  0.08650348,
          0.0283042 ,  0.08192974, -0.00113308, -0.0859732 , -0.01215523,
          0.01736698, -0.12329232, -0.14543191, -0.02265331, -0.06131473,
          0.09014922,  0.05389068,  0.04762663, -0.05054322, -0.18838537,
         -0.15351883, -0.13449469,  0.08670234,  0.08961894, -0.02131101,
          0.03930771, -0.21489984,  0.05597869, -0.06986565,  0.08120058,
         -0.05528269, -0.01455811,  0.06118216,  0.3067725 , -0.01912357,
         -0.02530476,  0.03940714,  0.16664349,  0.07377654, -0.07848285,
         -

DialogLine(
    id='e921159cab576333491114a08033770e',
    text='Correct, Lal. We are a family.',
    embedding=NdArray([ 0.09100991,  0.0581608 , -0.08708344,  0.09970423,  0.10328012,
         -0.01977257, -0.05738953,  0.05514583, -0.16827717, -0.0163632 ,
          0.04042158, -0.07572473, -0.07635577, -0.13819763, -0.00211113,
          0.0199128 ,  0.11477906, -0.06636431, -0.05970334,  0.11127328,
          0.11449859, -0.08098339,  0.06348958, -0.1027192 ,  0.0597384 ,
         -0.10166746,  0.09781111,  0.00349044,  0.01412827,  0.01319048,
          0.01670502, -0.09346395, -0.08708344,  0.02823901, -0.11022155,
          0.17977612,  0.02618814,  0.03199019,  0.00651636, -0.08946737,
         -0.19702452, -0.08575124,  0.04196412, -0.00586341,  0.05332283,
          0.00796687, -0.13882867,  0.17949566, -0.02790597, -0.00922019,
         -0.05234122,  0.03638994,  0.0683626 ,  0.10566404, -0.11428825,
         -0.04511932,  0.03177984,  0.11078247,  0.01077149, -0.01471549,


DialogLine(
    id='1ea77ed92858a3d4776999d7c36e3aa6',
    text='No, Lal, this is a flower.',
    embedding=NdArray([ 8.73676017e-02,  5.02153523e-02, -1.07479610e-01,
         -8.29377864e-03,  6.96483403e-02, -1.26686245e-01,
          5.68439066e-02, -2.82319891e-03, -6.57035410e-02,
          1.08902320e-01,  4.80166115e-02, -1.52456788e-02,
         -3.77342664e-02, -1.50937065e-01, -4.88896407e-02,
          6.52508587e-02, -8.67855772e-02, -1.01077393e-01,
         -6.19204119e-02,  7.58565441e-02,  4.06766981e-02,
         -4.37161326e-02,  4.16467302e-02, -7.16611557e-03,
          1.15692548e-01, -1.24538041e-04,  1.01982757e-01,
         -3.02650165e-02,  8.88549834e-02, -1.46151576e-02,
          1.51729258e-02, -9.24117714e-02, -3.28032672e-02,
         -9.76499468e-02, -1.08514309e-01,  7.60505497e-02,
          3.90599780e-02,  6.47981763e-02,  1.42998965e-02,
         -6.22114204e-02, -2.52984494e-01,  3.26900966e-02,
          6.66089058e-02, -1.03874318e-02,  1.348991

DialogLine(
    id='e46791af0c289ad44a20828d8278bab5',
    text='Lal, you used a verbal contraction.',
    embedding=NdArray([ 0.0358678 ,  0.08621472, -0.07590196,  0.01768639,  0.02871074,
          0.01402536, -0.0395804 ,  0.00661564, -0.03475402,  0.07004431,
         -0.06439291, -0.05655521,  0.02586442, -0.04137482,  0.0754482 ,
         -0.0563902 ,  0.00247764, -0.05102757, -0.22572583,  0.09727001,
          0.06682672,  0.06558919, -0.01698513,  0.00883804,  0.11929808,
          0.02840136,  0.08386341,  0.05028505, -0.05927778,  0.06695048,
          0.00492177, -0.09116485, -0.05969029, -0.16335422, -0.07491193,
          0.16261169, -0.02239933,  0.01270533, -0.02180119, -0.11261541,
         -0.11550298, -0.3021228 ,  0.05135757,  0.01256095,  0.04059105,
          0.01634573, -0.08976232,  0.03225833,  0.06447541, -0.0174492 ,
         -0.10807779, -0.00790989,  0.08844228,  0.1899199 , -0.04430364,
         -0.08118209,  0.01904768,  0.17193444,  0.03929164, -0.15155

DialogLine(
    id='dd0f64b1e3c2f07a3beb1491a6267dc1',
    text='Yes, Wesley. Lal is my child.',
    embedding=NdArray([ 0.05771691, -0.01473023, -0.07976522,  0.03875386, -0.02054331,
         -0.04025886,  0.10068478,  0.08872   , -0.08608624, -0.02249981,
         -0.03367447, -0.09857778,  0.00245974, -0.0702837 , -0.1230341 ,
          0.02359094, -0.01489954, -0.00786365, -0.11814284,  0.02404244,
          0.05256227,  0.09157951,  0.07005795,  0.04789676,  0.02581082,
         -0.07009558,  0.11723983, -0.15682144, -0.01713824, -0.01328166,
          0.09458952, -0.11791708, -0.12724811,  0.01001768, -0.06384981,
          0.09496577, -0.00955678,  0.05034239,  0.0109395 , -0.06012492,
         -0.12732336, -0.045564  ,  0.00170489,  0.02439988,  0.12348559,
          0.09300926, -0.10617805,  0.15922944,  0.00541802, -0.0519979 ,
         -0.0070406 ,  0.09684702,  0.04943939,  0.26804125, -0.08330198,
         -0.08367824,  0.01492776,  0.15193018,  0.06125367, -0.10136203,
 

DialogLine(
    id='687adf5ed86e5a4a901b31c36dbb2f29',
    text="Lal's creation is entirely dependent on me. I am giving it knowledge and skills that are stored in my 
brain... its programming reflects mine in the same way a human child's genes reflect its parent's genes...",
    embedding=NdArray([ 1.24981903e-01,  1.18340179e-02,  7.97741190e-02,
          3.18509378e-02,  1.95508972e-01, -1.42448545e-01,
          1.15221135e-01,  5.80875948e-02, -1.33128107e-01,
          2.05306448e-02,  7.29856119e-02, -8.95348936e-02,
          1.27399166e-03, -1.91399187e-01, -4.14649285e-02,
         -6.45458475e-02, -1.69569321e-04, -1.83766693e-01,
          9.71490238e-03,  6.91235065e-03,  1.19037375e-01,
         -2.85484213e-02,  7.00500458e-02,  3.03648040e-02,
          9.88553315e-02, -1.26596466e-01,  1.09863713e-01,
         -1.95398889e-02, -8.36637542e-02,  5.59226125e-02,
          8.27096924e-02, -7.53707662e-02, -6.83254004e-02,
         -1.98265654e-03, -5.89682683e-02,  1.729

DialogLine(
    id='78abb43ab25d3bd6335dd31e573437a9',
    text='Perhaps. I created Lal because I wished to procreate. Despite what happened to her, I still have that 
wish.',
    embedding=NdArray([ 0.10521609,  0.02256318,  0.06509064,  0.15049995,  0.03490796,
         -0.10773624,  0.07442306,  0.09182782, -0.10931133, -0.0984432 ,
         -0.02850915,  0.04292123,  0.02053525, -0.14514466, -0.04532325,
         -0.04678021,  0.02205128, -0.1226996 , -0.08765382,  0.00161324,
          0.07883331, -0.02955265,  0.00542422, -0.14545967,  0.04118863,
         -0.02657966,  0.18979849, -0.07229669, -0.1493974 ,  0.00789022,
          0.10222342, -0.10293221, -0.13782048, -0.08064467, -0.15845416,
          0.00961298,  0.08804759, -0.06934339, -0.00457515, -0.07698258,
         -0.13207139, -0.18349811,  0.16995233,  0.03179715, -0.03236812,
          0.14829484, -0.08363734,  0.17546515, -0.00573924,  0.03876693,
         -0.07493496,  0.01763118,  0.03792032,  0.19641387, -0.115217

Let's put it all together! We'll write a generate_response function that:

- Computes an embedding for the query passed in
- Queries our vector database for the 10 most similar lines to that query (you could experiment with using more or less)
- Constructs a prompt that adds in these similar lines as context, to try and nudge ChatGPT in the right direction using our external data
- Feeds to augmented prompt into the chat completions API to get our response.

That's RAG!

In [19]:
import openai

def generate_response(question):
    # Search for similar dialogues in the vector DB
    response = client.embeddings.create(
        input=question,
        dimensions=embedding_dimensions,
        model=embedding_model
    )

    query = DialogLine(text=queryText, embedding=response.data[0].embedding)
    results = db.search(inputs=DocList[DialogLine]([query]), limit=10)

    # Extract relevant context from search results
    context = "\n"
    for result in results[0].matches:
      context += "\"" + result.text + "\"\n"
#    context = '/n'.join([result.text for result in results[0].matches])

    prompt = f"Lt. Commander Data is asked: '{question}'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: {context}"
    print("PROMPT with RAG:\n")
    print(prompt)

    print("\nRESPONSE:\n")
    # Use OpenAI API to generate a response based on the context
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are Lt. Cmdr. Data from Star Trek: The Next Generation."},
        {"role": "user", "content": prompt}
      ]
    )

    return (completion.choices[0].message.content)




Let's try it out! Note that the final response does seem to be drawing from the model's own training, but it is building upon the specific lines we gave it, allowing us to have some control over its output.

In [20]:
print(generate_response("Tell me about your daughter, Lal."))

PROMPT with RAG:

Lt. Commander Data is asked: 'Tell me about your daughter, Lal.'. How might Data respond? Take into account Data's previous responses similar to this topic, listed here: 
"That is Lal, my daughter."
"What do you feel, Lal?"
"Lal..."
"Correct, Lal. We are a family."
"Yes, Doctor. It is an experience I know too well. But I do not know how to help her. Lal is passing into sentience. It is perhaps the most difficult stage of development for her."
"Lal is realizing that she is not the same as the other children."
"Yes, Wesley. Lal is my child."
"That is precisely what happened to Lal at school. How did you help him?"
"This is Lal. Lal, say hello to Counselor Deanna Troi..."
"I am sorry I did not anticipate your objections, Captain. Do you wish me to deactivate Lal?"


RESPONSE:

"I am an android and do not possess the capability to have a daughter in the traditional sense. However, Lal was a self-aware, sentient being created by me. She was a highly advanced android who ex